# Stock API to Excel

In [1]:
# basic imports for overall functionality
import numpy as np 
import pandas as pd
import requests
import ast
import http.client as httplib
from datetime import timedelta, date
from datetime import datetime

# Error handling
import traceback

# Display imports
from IPython.display import display, HTML

https://documentation.tradier.com/brokerage-api/markets/get-quotes

In [2]:
class API():
    
    def __init__(self, portfolio, key):
        self.portfolio = [p.upper() for p in list(sorted(set(portfolio)))]
        self.key = key
        
        self.start = str(pd.datetime.today() - timedelta(days=90)).split(" ")[0]
        self.end = str(pd.datetime.today() + timedelta(days=1)).split(" ")[0]
        self.now = str(pd.datetime.today()).split(" ")[0]
        
        self.loop_port = self.loop_port(self.portfolio, self.start, self.end, self.key)
    
    #API 
    def tradier(self, key, endpoint, par):
        connection = httplib.HTTPSConnection('sandbox.tradier.com', 443, timeout = 30)
        # Headers

        headers = {"Accept":"application/json",
               "Authorization":"Bearer {}".format(key)}

        # Send synchronously
        connection.request('GET', '{}{}'.format(endpoint, par), None, headers, encode_chunked=False)
        try:
          response = connection.getresponse()
          content = response.read()
          # Success
          print('Response status ' + str(response.status))

        except httplib.HTTPException:
            # Exception
            print('Exception during request')
        return content

    def b_decode(self, content):
        new_str = content.decode('utf-8') # Decode using the utf-8 encoding
        save = ast.literal_eval(new_str)
        col1 = [col for col in save.keys()][0]
        cut_down = save[col1]

        col2 = [col for col in cut_down.keys()][0]
        cut_down2 = cut_down[col2]

        df = pd.DataFrame(cut_down2)
        df.columns = df.columns.str.capitalize()
        return df
    
    def loop_port(self, portfolio, start, end, key):
        port = pd.DataFrame(columns=['Close', 'Date', 'High', 'Low', 'Open', 'Volume', 'Symbol'])
        slist = []
        for symbol in portfolio:

            content = self.tradier(key, '/v1/markets/history',
                      "?symbol={}&session_filter=all&start={}&end={}".format(symbol, start, end))
            try:
                file = pd.DataFrame(self.b_decode(content))
                file["Symbol"] = symbol
                slist.append(file)
                print(symbol)
            except:
                print(content, symbol)

        return slist

In [3]:
def prep(data, start, end):
    """Data that came straight from API class
    """
    hold = []
    for df in data.loop_port:
        
        df["Date"] = pd.to_datetime(df["Date"])
        
        df = df.loc[(df["Date"] >= pd.to_datetime(start)) & (df["Date"] <= pd.to_datetime(end))].reset_index(drop= True)
        df["Max"] = " "
        df["Min"] = " "
        
        
        df["Change"] = (df["Close"] - df["Close"].shift(1).round(2))
        df["Change %"] =  100 * (df["Close"] - df["Close"].shift(1).round(2)) / df["Close"].shift(1).round(2)
        
        df.loc[df["High"].index == df["High"].idxmax(), "Max"] = df["High"].max()
        df.loc[df["Low"].index == df["Low"].idxmin(), "Min"] = df["Low"].min()
         
        
        #display df
        display_df = df[::-1]
        display_df["Date"] = display_df["Date"].apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y'))

        symbol = df["Symbol"].values[0]
        
        #Hold the final df
        hold.append(display_df)
        
    return hold

### Start inputs

In [4]:
#API key
key = "<TOKEN>" # <---- Replace <TOKEN> with your Tradier key 

In [5]:
portfolio = ["AAPL", 'WMT', "SPY"] # <-- You can add more Symbols here

"""Change the dates below to change the range"""
start = "2023-02-28" # <-- You can change date
end = "2023-05-22" # <-- You can change date


In [6]:
data = API(portfolio, key)

C:\Users\mereb\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.

C:\Users\mereb\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.

C:\Users\mereb\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.



Response status 200
AAPL
Response status 200
SPY
Response status 200
WMT


In [7]:
stock_dfs = prep(data, start, end)

In [8]:
summary_list = [] 

#Creates Excel file
with pd.ExcelWriter('Stock API to Excel.xlsx') as writer: 
    
    
    for i in range(len(see)):
        ticker = see[i]["Symbol"][0]
        last_cell = len(see[i])+1
        
        
        max_value = [float(i) for i in list(set(see[i]["Max"])) if i !=  " " ][0]
        min_value = [float(i) for i in list(set(see[i]["Min"])) if i !=  " " ][0]
        summary_list.append([ticker, max_value, min_value])
        
        sum_df = pd.DataFrame(summary_list, columns =['Symb', "High", "Low"]) 
        sum_df.to_excel(writer, sheet_name="Summary")
        
        see[i].to_excel(writer, sheet_name=ticker)
        
        #add chart
        workbook  = writer.book
        worksheet = writer.sheets[ticker]
        #RED
        format1 = workbook.add_format({'bg_color': '#FFC7CE',
                               'font_color': '#9C0006'})
        #GREEN
        format2 = workbook.add_format({'bg_color': '#C6EFCE',
                               'font_color': '#006100'})

        #Max
        worksheet.conditional_format('D2:D{}'.format(last_cell), {'type': 'top',
                                         'value': '1',
                                         'format': format2})
        #Min
        worksheet.conditional_format('E2:E{}'.format(last_cell), {'type': 'bottom',
                                         'value': '1',
                                         'format': format1})
        
        
        chart = workbook.add_chart({'type': 'stock'})
        #Open
        chart.add_series({'categories': '={}!$B$2:$B${}'.format(ticker, last_cell),
                          'values': '={}!$C$2:$C${}'.format(ticker, last_cell),
                          'marker': {'type': 'none'}})
        #High
        chart.add_series({'categories': '={}!$B$2:$B${}'.format(ticker, last_cell),
                          'values': '={}!$D$2:$D${}'.format(ticker, last_cell)})
        #Low
        chart.add_series({'categories': '={}!$B$2:$B${}'.format(ticker, last_cell),
                          'values': '={}!$E$2:$E${}'.format(ticker, last_cell)})
        #Close
        chart.add_series({'categories': '={}!$B$2:$B${}'.format(ticker, last_cell),
                          'values': '={}!$F$2:$F${}'.format(ticker, last_cell),
                          'line': {'none': True},
                          'marker': {'type': 'none'}})
        chart.set_legend({'none': True})

        chart.set_title ({'name': ticker})
        chart.set_x_axis({'date_axis': True, 'reverse': True})
        chart.set_y_axis({
            'min': min_value,
            'max': max_value
        })
        chart.set_up_down_bars({
            'up': {
                'fill':   {'color': '#92D050'},
                'border': {'color': 'black'}
            },
            'down': {
                'fill':   {'color': '#FF0000'},
                'border': {'color': 'black'},
            },
        })
        worksheet.insert_chart('M6', chart)

writer.save() 
writer.close()

[          Date     Open      High       Low    Close     Volume Symbol  \
 58  05/22/2023  173.980  174.7100  173.4500  174.200   43501674   AAPL   
 57  05/19/2023  176.390  176.3900  174.9400  175.160   55809475   AAPL   
 56  05/18/2023  173.000  175.2400  172.5800  175.050   65496657   AAPL   
 55  05/17/2023  171.710  172.9250  170.4201  172.690   57951604   AAPL   
 54  05/16/2023  171.990  173.1383  171.7991  172.070   42110293   AAPL   
 53  05/15/2023  173.160  173.2100  171.4700  172.070   37266659   AAPL   
 52  05/12/2023  173.620  174.0600  171.0000  172.570   45533138   AAPL   
 51  05/11/2023  173.850  174.5900  172.1700  173.750   49514676   AAPL   
 50  05/10/2023  173.020  174.0300  171.9000  173.555   53724501   AAPL   
 49  05/09/2023  173.050  173.5400  171.6000  171.770   45326874   AAPL   
 48  05/08/2023  172.480  173.8500  172.1100  173.500   55962793   AAPL   
 47  05/05/2023  170.975  174.3000  170.7600  173.570  113453171   AAPL   
 46  05/04/2023  164.890 